In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from utils_mri import get_itk_array, get_itk_image, write_itk_imageArray

def calculate_volume(data, atlas, label):
    roi = (atlas == label).astype('uint8')
    volume = np.sum(data * roi)
    return volume

def calculate_structure_volume(data, atlas, labels):
    label_dict = {}
    for i, structure in labels.iterrows():
        if structure['left label'] == structure['right label']:
            label_dict.update({structure['Structure']: calculate_volume(data, atlas, structure['left label'])})
        else:
            label_dict.update({
                structure['Structure'] + '_left': calculate_volume(data, atlas, structure['left label']),
                structure['Structure'] + '_right': calculate_volume(data, atlas, structure['right label']),
            })
    return label_dict

def refine_atlas(atlas, labels):
    new_atlas = np.zeros_like(atlas)
    for i, structure in labels.iterrows():
        if structure['left label'] == structure['right label']:
            new_atlas += (atlas == structure['left label']).astype('uint8') * structure['left label']
        else:
            new_atlas += (atlas == structure['left label']).astype('uint8') * structure['left label']
            new_atlas += (atlas == structure['right label']).astype('uint8') * structure['right label']
    return new_atlas

In [2]:
root = 'E:/FDU/ISTBI/fMRI/'

In [3]:
labels = pd.read_csv(os.path.join(root, 'fd_Dorr/c57_brain_atlas_labels.csv'), index_col=0)
atlas = get_itk_array(os.path.join(root, 'fd_Dorr/Dorr_Turone_brain/dorr2turone_labels_refine.nii'))

In [4]:
scan_info = pd.read_excel(os.path.join(root, 'scan_info.xlsx'))
structure_info = scan_info[scan_info['ScanType'] == 'T1']

In [5]:
info_list = []
for i, info in structure_info.iterrows():
    MouseIDSub = 'sub{:0>4d}'.format(info['MouseID'])
    c1_path = os.path.join(root, 'pm2.5-offspring-mouse/{}/T2/mwc1mT2.nii'.format(MouseIDSub))
    c1 = get_itk_array(c1_path)
    label_dict = calculate_structure_volume(c1, atlas, labels)
    new_info = dict(info)
    new_info.update(label_dict)
    info_list.append(new_info)

In [6]:
info_pd = pd.DataFrame(info_list)

In [7]:
info_pd.to_excel('structure_info.xlsx')

In [8]:
# new_atlas = refine_atlas(atlas, labels)
# write_itk_imageArray(new_atlas, os.path.join(root, 'fd_Dorr/Dorr_Turone_brain/dorr2turone_labels_refine_refine.nii'), get_itk_image(os.path.join(root, 'fd_Dorr/Dorr_Turone_brain/dorr2turone_labels_refine.nii')))